***HW2 - Detecting Type 1 Diabetes***

# Theory Assignment

**A1**

For our opinion, when it comes to healthcare ML algorithms, Accuracy is more important than performance.
Because we're speaking on human lives, it is better to be more precise than, for example, when estimating house value.

האם התכוונו כאן לביצועיות של המודל? כלומר למהירות בה הוא מגיע לתוצאות?

**A2**

Too many features is often a bad thing. It may lead to Overfitting, meaning that the fitting of your parameters is too tightly to the training data. This results in model discovering random noise in the finite training set instead of the wider relationship between the features and the output variable. Consequently, the model will often perform very well on the training data but perform quite poorly on the test data.
In conclusion, overfitting may be done due to choosing all features, especially whenn taking irrelevant feature like income.

On the other hand, choosing only 2 features for such complex problem may cause under-fitting. meaning that both training accuracy and testing accuracy will be poor.


# Coding Assignment:

Import and load packages:

In [ ]:
import pandas as pd 
import numpy as np
from pathlib import Path
import random
import distutils
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from Ext_Functions import str_to_bool_series

# **Q1 - Data loading and preprocessing:**

Load data:

In [ ]:
df_orig = pd.read_csv('HW2_data.csv')

Transfer all data to numeric values:

In [ ]:
from Ext_Functions import nan2num

df = df_orig
for column in df.columns:
    df[column] = str_to_bool_series(df_orig[column])
df.tail()

Clean NaN's:


In [ ]:
t1d_clean = nan2num(df)
t1d_clean

Scale Age column:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
t1d_clean[['Age']] = min_max_scaler.fit_transform(t1d_clean[['Age']])
t1d_clean

# **Q2 -  Split the data into Test Train 20%**

In [ ]:
diagnosis = t1d_clean['Diagnosis']

X_train, X_test, y_train, y_test = train_test_split(t1d_clean, np.ravel(diagnosis), test_size=0.2,
                                                        random_state=0, stratify=np.ravel(diagnosis))
X_test

# **Q3 - Visualization and exploration of the data**



# Distribution of the features:

In [ ]:
from Ext_Functions import print_dist_features
print_dist_features(t1d_clean, X_train, X_test)

**A3 Section a**

Imbalance between train and test set could cause the learning algorithm to fail in classification of the test set.
Balance can be reached with stratification or normalization. in our case, due to the booleanic nature of the data, its better to do stratification.

# Relationship between feature and label:

These plots shows the relationsship between each feature and label. The age feature is shown in zoom-in plot because it has many details and was not clear in a small plot

In [ ]:
import seaborn as sbn
import matplotlib.ticker as ticker
from Ext_Functions import feature_corr

feature_corr(t1d_clean)

fig, axes = plt.subplots(1, 1,figsize=(15, 10) )


# Plot Age seperately:
title_text = 'Number of patients as a function of age - Zoom in'
feat_lab = sbn.countplot(ax = axes, x='Age', hue = 'Diagnosis', data = t1d_clean)
feat_lab.xaxis.set_major_locator(ticker.LinearLocator(10))
feat_lab.set_title(title_text, fontsize=15)
axes.tick_params(axis='both', which='major', labelsize=15)

Additional plots:

In [ ]:
X_train = X_train.drop(columns=['Diagnosis'])
X_test = X_test.drop(columns=['Diagnosis'])

# **Q4 - One Hot Vector**

**We have already done this step - manually - in the data pre-processing stage**

In [ ]:
X_train_ohv = X_train
X_test_ohv = X_test
y_train_ohv = y_train
y_test_ohv = y_test

(X_train_ohv)

Additional plots:

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import NuSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
from Ext_Functions import cv_kfold
from Ext_Functions import pred_log

**Test and Fit the models**

In [ ]:
C = np.array([0.01, 0.1, 1, 5, 10, 100]) 
K = 5  
penalty = ['l1', 'l2']
val_dict = cv_kfold(X_train_ohv, y_train_ohv, C=C, penalty=penalty, K=K)
print(pd.DataFrame(val_dict))

# Insert best parameters to get the best model:

c = 5
p = 'l1'
lr_best_model = LogisticRegression(solver='saga', multi_class='ovr', penalty=p, C=c, max_iter=10000, random_state=10)
y_pred_best, _ = pred_log(lr_best_model, X_train_ohv, y_train_ohv, X_test_ohv)
y_pred_p_best, _ = pred_log(lr_best_model, X_train_ohv, y_train_ohv, X_test_ohv, flag=True)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

n_splits = K
skfold = StratifiedKFold(n_splits=n_splits, random_state=10, shuffle=True)
svc_model = SVC(probability=True)
C = np.array([1, 100, 1000])
pipeline = Pipeline(steps=[('scale', StandardScaler()), ('svm', svc_model)])
svm_nonlinear = GridSearchCV(estimator=pipeline, param_grid={'svm__C': C, 'svm__kernel': ['rbf', 'poly'],
                        'svm__gamma': ['auto', 'scale']}, scoring=['roc_auc'],
                        cv=skfold, refit='roc_auc', verbose=0, return_train_score=True)
svm_nonlinear.fit(X_train_ohv, y_train_ohv)

best_svm_nonlin = svm_nonlinear.best_estimator_
print('Parameters to get best svm are:', svm_nonlinear.best_params_)

In [ ]:
from sklearn.metrics import plot_confusion_matrix


disp_lr = plot_confusion_matrix(lr_best_model, X_test_ohv, y_test_ohv,
                                 display_labels=['Negative', 'Positive'],
                                 cmap=plt.cm.Blues)
disp_lr.ax_.set_title("Confusion Matrix - Logistic Regression")

disp_svm = plot_confusion_matrix(best_svm_nonlin, X_test_ohv, y_test_ohv,
                                 display_labels=['Negative', 'Positive'],
                                 cmap=plt.cm.Blues)
disp_svm.ax_.set_title("Confusion Matrix - Non-linear SVM")

plt.show()

# Q5 part b

Plot statistics for all algorithms:

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from Ext_Functions import parameter_evaluation
# lr = Logistic Regression
# svm = Support Vector Machine


model_lr = lr_best_model
model_svm = best_svm_nonlin
y_pred_lr = model_lr.decision_function(X_test_ohv)
lr_fpr, lr_tpr, threshold = roc_curve(y_test_ohv, y_pred_lr)
y_pred_lr_train = (model_lr.predict(X_train_ohv)).astype("int32")
y_pred_lr_test = (model_lr.predict(X_test_ohv)).astype("int32")
auc_lr = metrics.roc_auc_score(y_test,y_pred_lr_test)


y_pred_svm = model_svm.decision_function(X_test_ohv)
svm_fpr, svm_tpr, threshold = roc_curve(y_test_ohv, y_pred_svm)
y_pred_svm_train = (model_svm.predict(X_train_ohv)).astype("int32")
y_pred_svm_test = (model_svm.predict(X_test_ohv)).astype("int32")
auc_svm = metrics.roc_auc_score(y_test,y_pred_svm_test)


plt.figure(figsize = (5,5), dpi = 100)
plt.plot(svm_fpr, svm_tpr, linestyle = '-', label = 'SVM (auc=%0.3f)' %auc_svm)
plt.plot(lr_fpr, lr_tpr,marker='.' ,label = 'Logistic Reg (auc=%0.3f)' %auc_lr)
plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')
plt.legend()
plt.show()

print('Logistic Regression Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_lr_test,y_pred_lr_train)

print()
print('SVM Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_svm_test,y_pred_svm_train)

# **Q6 - Feature importance**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X_train,y_train)
# get importance
importance = model_rf.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
 	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
labels = ['Age','Gender_M', 'Increased Urination_T', 'Increased Thirst_T',
       'Sudden Weight Loss_T', 'Weakness_T',
        'Increased Hunger_T','Genital Thrush_T', 'Visual Blurring_T',  
        'Itching_T', 'Irritability_T','Delayed Healing_T',
        'Partial Paresis_T', 'Muscle Stiffness_T', 'Hair Loss_T',
        'Obesity_T','Family History_T']
t= np.arange(0,17)
plt.bar([x for x in range(len(importance))], importance)
plt.xticks(t,labels, rotation='vertical')
plt.show()

As we can see, the 2 most important features are increased urination and increased thirst.

This matches up with the feature - label correlation from Q3. Looking at the graphs, we can see that these features are the most meaningful ones since they have the largest number of diagnosed patients. Also, among the patients who do have increased urination\thirst, the number of the healthy ones is the lowest in comparison to the diagnsed ones.

# Q7 - Data Separability Visualization

Section a

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
n_components = 2
pca = PCA(n_components=n_components, whiten=True)
scaler = StandardScaler()
X_train_copy = scaler.fit_transform(X_train_ohv)
X_test_copy = scaler.transform(X_test_ohv)
X_train_pca = pca.fit_transform(X_train_copy)
X_test_pca = pca.transform(X_test_copy)

In [ ]:
def plt_2d_pca(X_pca,y):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, aspect='equal')
    ax.scatter(X_pca[y==0, 0], X_pca[y==0, 1], color='b')
    ax.scatter(X_pca[y==1, 0], X_pca[y==1, 1], color='r')
    ax.legend(('Negative','Positive'))
    ax.plot([0], [0], "ko")
    ax.arrow(0, 0, 0, 1, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
    ax.arrow(0, 0, 1, 0, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
    ax.set_xlabel('$U_1$')
    ax.set_ylabel('$U_2$')
    ax.set_title('2D PCA')

In [ ]:
plt_2d_pca(X_test_pca,y_test)

Section b

As can be seen above, the data is not linearly seperable. It is possible to use a linear classifier, however the boundary condition may be misleading.

In [ ]:
from sklearn.metrics import plot_confusion_matrix, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import hinge_loss
calc_TN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 0]
calc_FP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 1]
calc_FN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 0]
calc_TP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 1]


Finding the perfect parameters for the new-2D-data:

In [ ]:
#Linear Regression:

X_train_pca = pd.DataFrame(X_train_pca)
X_test_pca = pd.DataFrame(X_test_pca)

C = np.array([0.01, 0.1, 1, 5, 10, 100]) 
K = 5  
penalty = ['l1', 'l2']
val_dict_pca = cv_kfold(X_train_pca, y_train_ohv, C=C, penalty=penalty, K=K)
print(pd.DataFrame(val_dict_pca))

# Insert best parameters to get the best model (PCA):

c = 1
p = 'l2'
lr_best_model_pca = LogisticRegression(solver='saga', multi_class='ovr', penalty=p, C=c, max_iter=10000, random_state=10)
y_pred_best_pca, _ = pred_log(lr_best_model_pca, X_train_pca, y_train_ohv, X_test_pca)
y_pred_prob_best_pca, _ = pred_log(lr_best_model_pca, X_train_pca, y_train_ohv, X_test_pca, flag=True)

In [ ]:
n_splits = K
skfold = StratifiedKFold(n_splits=n_splits, random_state=10, shuffle=True)
svc_model = SVC(probability=True)
C = np.array([1, 100, 1000])
pipeline = Pipeline(steps=[('scale', StandardScaler()), ('svm', svc_model)])
svm_nonlinear_pca = GridSearchCV(estimator=pipeline, param_grid={'svm__C': C, 'svm__kernel': ['rbf', 'poly'],
                        'svm__gamma': ['auto', 'scale']}, scoring=['roc_auc'],
                        cv=skfold, refit='roc_auc', verbose=0, return_train_score=True)
svm_nonlinear_pca.fit(X_train_pca, y_train_ohv)

best_svm_nonlin_pca = svm_nonlinear_pca.best_estimator_
print('Parameters to get best svm are:', svm_nonlinear_pca.best_params_)

In [ ]:
#Predict + fit for LR and SVM (PCA):

lr_best_model_pca.fit(X_train_pca, y_train)
y_pred_test_pca = lr_best_model_pca.predict(X_test_pca)

model_lr_pca = lr_best_model_pca
model_svm_pca = best_svm_nonlin_pca

y_pred_lr_pca = model_lr_pca.decision_function(X_test_pca)
lr_fpr_pca, lr_tpr_pca, threshold_pca = roc_curve(y_test_ohv, y_pred_lr_pca)
y_pred_lr_train_pca = (model_lr_pca.predict(X_train_pca)).astype("int32")
y_pred_lr_test_pca = (model_lr_pca.predict(X_test_pca)).astype("int32")
auc_lr_pca = metrics.roc_auc_score(y_test,y_pred_lr_test_pca)


y_pred_svm_pca = best_svm_nonlin_pca.decision_function(X_test_pca)
svm_fpr_pca, svm_tpr_pca, threshold_pca = roc_curve(y_test_ohv, y_pred_svm_pca)
y_pred_svm_train_pca = (best_svm_nonlin_pca.predict(X_train_pca)).astype("int32")
y_pred_svm_test_pca = (best_svm_nonlin_pca.predict(X_test_pca)).astype("int32")
auc_svm_pca = metrics.roc_auc_score(y_test,y_pred_svm_test_pca)


plt.figure(figsize = (5,5), dpi = 100)
plt.plot(svm_fpr_pca, svm_tpr_pca, linestyle = '-', label = 'SVM - 2D (auc=%0.3f)' %auc_svm_pca)
plt.plot(lr_fpr_pca, lr_tpr_pca,marker='.' ,label = 'Logistic Reg - 2D(auc=%0.3f)' %auc_lr_pca)
plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')
plt.legend()
plt.show()

print('Logistic Regression Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_lr_test_pca,y_pred_lr_train_pca)

print()
print('SVM Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_svm_test_pca,y_pred_svm_train_pca)

Section D:

In [ ]:
feat_name =['Increased Urination','Increased Thirst']
X_train_2f = X_train[feat_name]
X_test_2f =  X_test[feat_name]

Again - finding the best parameters for the new data:

In [ ]:
#Linear Regression:

X_train_2f = pd.DataFrame(X_train_2f)
X_test_2f = pd.DataFrame(X_test_2f)

C = np.array([0.01, 0.1, 1, 5, 10, 100]) 
K = 5  
penalty = ['l1', 'l2']
val_dict_2f = cv_kfold(X_train_2f, y_train_ohv, C=C, penalty=penalty, K=K)
print(pd.DataFrame(val_dict_2f))

# Insert best parameters to get the best model (PCA):

c = 0.1
p = 'l2'
lr_best_model_2f = LogisticRegression(solver='saga', multi_class='ovr', penalty=p, C=c, max_iter=10000, random_state=10)
y_pred_best_2f, _ = pred_log(lr_best_model_2f, X_train_2f, y_train_ohv, X_test_2f)
y_pred_prob_best_2f, _ = pred_log(lr_best_model_2f, X_train_2f, y_train_ohv, X_test_pca, flag=True)

In [ ]:
n_splits = K
skfold = StratifiedKFold(n_splits=n_splits, random_state=10, shuffle=True)
svc_model_2f = SVC(probability=True)
C = np.array([1, 100, 1000])
pipeline = Pipeline(steps=[('scale', StandardScaler()), ('svm', svc_model_2f)])
svm_nonlinear_2f = GridSearchCV(estimator=pipeline, param_grid={'svm__C': C, 'svm__kernel': ['rbf', 'poly'],
                        'svm__gamma': ['auto', 'scale']}, scoring=['roc_auc'],
                        cv=skfold, refit='roc_auc', verbose=0, return_train_score=True)
svm_nonlinear_2f.fit(X_train_2f, y_train_ohv)

best_svm_nonlin_2f = svm_nonlinear_2f.best_estimator_
print('Parameters to get best svm are:', svm_nonlinear_2f.best_params_)

In [ ]:
#Predict + fit for LR and SVM (PCA):

lr_best_model_2f.fit(X_train_2f, y_train)
y_pred_test_2f = lr_best_model_2f.predict(X_test_2f)

model_lr_2f = lr_best_model_2f
model_svm_2f = best_svm_nonlin_2f

y_pred_lr_2f = lr_best_model_2f.decision_function(X_test_2f)
lr_fpr_2f, lr_tpr_2f, threshold_2f = roc_curve(y_test_ohv, y_pred_lr_2f)
y_pred_lr_train_2f = (lr_best_model_2f.predict(X_train_2f)).astype("int32")
y_pred_lr_test_2f = (model_lr_2f.predict(X_test_2f)).astype("int32")
auc_lr_2f = metrics.roc_auc_score(y_test,y_pred_lr_test_2f)


y_pred_svm_2f = svm_nonlinear_2f.decision_function(X_test_2f)
svm_fpr_2f, svm_tpr_2f, threshold_2f = roc_curve(y_test_ohv, y_pred_svm_2f)
y_pred_svm_train_2f = (svm_nonlinear_2f.predict(X_train_2f)).astype("int32")
y_pred_svm_test_2f = (svm_nonlinear_2f.predict(X_test_2f)).astype("int32")
auc_svm_2f = metrics.roc_auc_score(y_test,y_pred_svm_test_2f)


plt.figure(figsize = (5,5), dpi = 100)
plt.plot(svm_fpr_2f, svm_tpr_2f, linestyle = '-', label = 'SVM - 2D (auc=%0.3f)' %auc_svm_2f)
plt.plot(lr_fpr_2f, lr_tpr_2f,marker='.' ,label = 'Logistic Reg - 2D(auc=%0.3f)' %auc_lr_2f)
plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')
plt.legend()
plt.show()

print('Logistic Regression Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_lr_test_2f,y_pred_lr_train_2f)

print()
print('SVM Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_svm_test_2f,y_pred_svm_train_2f)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
n_components = 2
pca = PCA(n_components=n_components, whiten=True)
scaler = StandardScaler()
X_train_copy = scaler.fit_transform(X_train_ohv)
X_test_copy = scaler.transform(X_test_ohv)
X_train_pca = pca.fit_transform(X_train_copy)
X_test_pca = pca.transform(X_test_copy)

In [ ]:
def plt_2d_pca(X_pca,y):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, aspect='equal')
    ax.scatter(X_pca[y==0, 0], X_pca[y==0, 1], color='b')
    ax.scatter(X_pca[y==1, 0], X_pca[y==1, 1], color='r')
    ax.legend(('Negative','Positive'))
    ax.plot([0], [0], "ko")
    ax.arrow(0, 0, 0, 1, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
    ax.arrow(0, 0, 1, 0, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
    ax.set_xlabel('$U_1$')
    ax.set_ylabel('$U_2$')
    ax.set_title('2D PCA')

In [ ]:
plt_2d_pca(X_test_pca,y_test)

Section b

As can be seen above, the data is not linearly seperable. It is possible to use a linear classifier, however the boundary condition may be misleading.

In [ ]:
from sklearn.metrics import plot_confusion_matrix, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import hinge_loss
calc_TN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 0]
calc_FP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 1]
calc_FN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 0]
calc_TP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 1]


Finding the perfect parameters for the new-2D-data:

In [ ]:
#Linear Regression:

X_train_pca = pd.DataFrame(X_train_pca)
X_test_pca = pd.DataFrame(X_test_pca)

C = np.array([0.01, 0.1, 1, 5, 10, 100]) 
K = 5  
penalty = ['l1', 'l2']
val_dict_pca = cv_kfold(X_train_pca, y_train_ohv, C=C, penalty=penalty, K=K)
print(pd.DataFrame(val_dict_pca))

# Insert best parameters to get the best model (PCA):

c = 1
p = 'l2'
lr_best_model_pca = LogisticRegression(solver='saga', multi_class='ovr', penalty=p, C=c, max_iter=10000, random_state=10)
y_pred_best_pca, _ = pred_log(lr_best_model_pca, X_train_pca, y_train_ohv, X_test_pca)
y_pred_prob_best_pca, _ = pred_log(lr_best_model_pca, X_train_pca, y_train_ohv, X_test_pca, flag=True)

In [ ]:
n_splits = K
skfold = StratifiedKFold(n_splits=n_splits, random_state=10, shuffle=True)
svc_model = SVC(probability=True)
C = np.array([1, 100, 1000])
pipeline = Pipeline(steps=[('scale', StandardScaler()), ('svm', svc_model)])
svm_nonlinear_pca = GridSearchCV(estimator=pipeline, param_grid={'svm__C': C, 'svm__kernel': ['rbf', 'poly'],
                        'svm__gamma': ['auto', 'scale']}, scoring=['roc_auc'],
                        cv=skfold, refit='roc_auc', verbose=0, return_train_score=True)
svm_nonlinear_pca.fit(X_train_pca, y_train_ohv)

best_svm_nonlin_pca = svm_nonlinear_pca.best_estimator_
print('Parameters to get best svm are:', svm_nonlinear_pca.best_params_)

In [ ]:
#Predict + fit for LR and SVM (PCA):

lr_best_model_pca.fit(X_train_pca, y_train)
y_pred_test_pca = lr_best_model_pca.predict(X_test_pca)

model_lr_pca = lr_best_model_pca
model_svm_pca = best_svm_nonlin_pca

y_pred_lr_pca = model_lr_pca.decision_function(X_test_pca)
lr_fpr_pca, lr_tpr_pca, threshold_pca = roc_curve(y_test_ohv, y_pred_lr_pca)
y_pred_lr_train_pca = (model_lr_pca.predict(X_train_pca)).astype("int32")
y_pred_lr_test_pca = (model_lr_pca.predict(X_test_pca)).astype("int32")
auc_lr_pca = metrics.roc_auc_score(y_test,y_pred_lr_test_pca)


y_pred_svm_pca = best_svm_nonlin_pca.decision_function(X_test_pca)
svm_fpr_pca, svm_tpr_pca, threshold_pca = roc_curve(y_test_ohv, y_pred_svm_pca)
y_pred_svm_train_pca = (best_svm_nonlin_pca.predict(X_train_pca)).astype("int32")
y_pred_svm_test_pca = (best_svm_nonlin_pca.predict(X_test_pca)).astype("int32")
auc_svm_pca = metrics.roc_auc_score(y_test,y_pred_svm_test_pca)


plt.figure(figsize = (5,5), dpi = 100)
plt.plot(svm_fpr_pca, svm_tpr_pca, linestyle = '-', label = 'SVM - 2D (auc=%0.3f)' %auc_svm_pca)
plt.plot(lr_fpr_pca, lr_tpr_pca,marker='.' ,label = 'Logistic Reg - 2D(auc=%0.3f)' %auc_lr_pca)
plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')
plt.legend()
plt.show()

print('Logistic Regression Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_lr_test_pca,y_pred_lr_train_pca)

print()
print('SVM Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_svm_test_pca,y_pred_svm_train_pca)

Section D:

In [ ]:
feat_name =['Increased Urination','Increased Thirst']
X_train_2f = X_train[feat_name]
X_test_2f =  X_test[feat_name]

Again - finding the best parameters for the new data:

In [ ]:
#Linear Regression:

X_train_2f = pd.DataFrame(X_train_2f)
X_test_2f = pd.DataFrame(X_test_2f)

C = np.array([0.01, 0.1, 1, 5, 10, 100]) 
K = 5  
penalty = ['l1', 'l2']
val_dict_2f = cv_kfold(X_train_2f, y_train_ohv, C=C, penalty=penalty, K=K)
print(pd.DataFrame(val_dict_2f))

# Insert best parameters to get the best model (PCA):

c = 0.1
p = 'l2'
lr_best_model_2f = LogisticRegression(solver='saga', multi_class='ovr', penalty=p, C=c, max_iter=10000, random_state=10)
y_pred_best_2f, _ = pred_log(lr_best_model_2f, X_train_2f, y_train_ohv, X_test_2f)
y_pred_prob_best_2f, _ = pred_log(lr_best_model_2f, X_train_2f, y_train_ohv, X_test_pca, flag=True)

In [ ]:
n_splits = K
skfold = StratifiedKFold(n_splits=n_splits, random_state=10, shuffle=True)
svc_model_2f = SVC(probability=True)
C = np.array([1, 100, 1000])
pipeline = Pipeline(steps=[('scale', StandardScaler()), ('svm', svc_model_2f)])
svm_nonlinear_2f = GridSearchCV(estimator=pipeline, param_grid={'svm__C': C, 'svm__kernel': ['rbf', 'poly'],
                        'svm__gamma': ['auto', 'scale']}, scoring=['roc_auc'],
                        cv=skfold, refit='roc_auc', verbose=0, return_train_score=True)
svm_nonlinear_2f.fit(X_train_2f, y_train_ohv)

best_svm_nonlin_2f = svm_nonlinear_2f.best_estimator_
print('Parameters to get best svm are:', svm_nonlinear_2f.best_params_)

In [ ]:
#Predict + fit for LR and SVM (PCA):

lr_best_model_2f.fit(X_train_2f, y_train)
y_pred_test_2f = lr_best_model_2f.predict(X_test_2f)

model_lr_2f = lr_best_model_2f
model_svm_2f = best_svm_nonlin_2f

y_pred_lr_2f = lr_best_model_2f.decision_function(X_test_2f)
lr_fpr_2f, lr_tpr_2f, threshold_2f = roc_curve(y_test_ohv, y_pred_lr_2f)
y_pred_lr_train_2f = (lr_best_model_2f.predict(X_train_2f)).astype("int32")
y_pred_lr_test_2f = (model_lr_2f.predict(X_test_2f)).astype("int32")
auc_lr_2f = metrics.roc_auc_score(y_test,y_pred_lr_test_2f)


y_pred_svm_2f = svm_nonlinear_2f.decision_function(X_test_2f)
svm_fpr_2f, svm_tpr_2f, threshold_2f = roc_curve(y_test_ohv, y_pred_svm_2f)
y_pred_svm_train_2f = (svm_nonlinear_2f.predict(X_train_2f)).astype("int32")
y_pred_svm_test_2f = (svm_nonlinear_2f.predict(X_test_2f)).astype("int32")
auc_svm_2f = metrics.roc_auc_score(y_test,y_pred_svm_test_2f)


plt.figure(figsize = (5,5), dpi = 100)
plt.plot(svm_fpr_2f, svm_tpr_2f, linestyle = '-', label = 'SVM - 2D (auc=%0.3f)' %auc_svm_2f)
plt.plot(lr_fpr_2f, lr_tpr_2f,marker='.' ,label = 'Logistic Reg - 2D(auc=%0.3f)' %auc_lr_2f)
plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')
plt.legend()
plt.show()

print('Logistic Regression Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_lr_test_2f,y_pred_lr_train_2f)

print()
print('SVM Results:')
parameter_evaluation(y_test_ohv,y_train_ohv,y_pred_svm_test_2f,y_pred_svm_train_2f)